## Modelo em produção (continuação Estudo_Airbnb.ipynb)

**Primeiro vamos trazer a base que queremos usar para fazer a previsão**

In [1]:
# Importando o pandas
import pandas as pd

In [11]:
# Importando a base que queremos analisar
base = pd.read_csv('producao.csv', sep = ';')
base2 = base # juntando as bases por conta da coluna 'aluguel' 

In [12]:
# Visualizando a base
base.head()

,bairro,condominio,iptu,metragem,quarto,banheiro,vaga_carro,andar,aceita_pet,mobilia,metro_prox,aluguel
0,Perdizes,448.0,34.0,58.0,1.0,1.0,1.0,6.0,1.0,1.0,0.0,NaN
1,Bela Vista,1106.0,218.0,74.0,3.0,2.0,1.0,5.0,1.0,1.0,0.0,NaN
2,Mooca,50.0,83.0,80.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN
3,Santana,230.0,50.0,120.0,2.0,2.0,1.0,2.0,1.0,0.0,1.0,NaN
4,Vila Mariana,1040.0,140.0,85.0,2.0,1.0,2.0,18.0,0.0,0.0,0.0,NaN


**Todos os tratamentos que fizemos na base do modelo, vamos precisar fazer para as bases de produção**<br>
**Já não temos as colunas seguro_incendio,taxa_serviço e total nessa base, falta só tratar os bairros**

In [13]:
# Importando o arquivo da região
regiao = pd.read_excel('bairros.xlsx')

In [14]:
# Unindo essas duas bases
base = pd.merge(
    base,
    regiao,
    how='left',
    left_on='bairro',right_on='Bairro'
)

In [15]:
# Eliminando as colunas de bairro e Bairro
base = base.drop(['bairro','Bairro'],axis=1)

In [16]:
# Já fazendo o One Hot Encoding (O One Hot Encoding é uma técnica de pré-processamento de dados que transforma variáveis categóricas em vetores binários)
base = pd.concat([base,pd.get_dummies(base.Região)],axis=1)

In [17]:
# E eliminando a coluna Região
base = base.drop('Região',axis=1)

In [18]:
# Visualizando a base
base.head(3)

,condominio,iptu,metragem,quarto,banheiro,vaga_carro,andar,aceita_pet,mobilia,metro_prox,aluguel,Centro,Leste,Norte,Oeste,Sul
0,448.0,34.0,58.0,1.0,1.0,1.0,6.0,1.0,1.0,0.0,NaN,0,0,0,1,0
1,1106.0,218.0,74.0,3.0,2.0,1.0,5.0,1.0,1.0,0.0,NaN,1,0,0,0,0
2,50.0,83.0,80.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,0,1,0,0,0


**Agora vamos recuperar o modelo que criamos**

In [27]:
# Importando o load do joblib
from joblib import load

In [29]:
# Carregando o nosso regressor
reg = load('regressor.joblib');

In [30]:
# Separando X e y
X = base.drop('aluguel',axis=1)
y = base.aluguel

In [31]:
# Fazendo a previsão para esses novos dados
y_pred = reg.predict(X)

In [32]:
# Visualizando y
y_pred

array([2834.78154765, 3818.18415128, 1764.16823607, 2838.16548462,
       2471.20682593])

In [33]:
# Podemos substituir o aluguel na nossa base inicial por esse valor de y_pred
base2['aluguel'] = y_pred

In [34]:
# Visualizando novamente a base
base2.head()

,bairro,condominio,iptu,metragem,quarto,banheiro,vaga_carro,andar,aceita_pet,mobilia,metro_prox,aluguel
0,Perdizes,448.0,34.0,58.0,1.0,1.0,1.0,6.0,1.0,1.0,0.0,2834.781548
1,Bela Vista,1106.0,218.0,74.0,3.0,2.0,1.0,5.0,1.0,1.0,0.0,3818.184151
2,Mooca,50.0,83.0,80.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1764.168236
3,Santana,230.0,50.0,120.0,2.0,2.0,1.0,2.0,1.0,0.0,1.0,2838.165485
4,Vila Mariana,1040.0,140.0,85.0,2.0,1.0,2.0,18.0,0.0,0.0,0.0,2471.206826


In [35]:
# Agora podemos exportar essa base para entregar como o resultado da nossa previsão
base2.to_excel('resultado.xlsx',index=False)